# Parametric Priors




Tomaremos como ejemplo de aplicación de los metódos de shrinkage el modelo de ajuste lineal $Y = \beta^{T} X$. El modelo lineal es en general una primera opción para unaa regresión dada sus simpleza y que es facil de interpretar. Sin embargo uno suele incorporar un gran número de variables que quizás sobre parametrizan el problema, y en ese caso las inferencias pueden resultar poco satisfactorias dado que suceden dos cosas:
1) Al introducir más parámetros, tenemos mayor incerteza al hacer las posteriors predictivas y por ende mayor varianza en las predicciones. 
2) El hecho de que tengamos muchos parámeros hace que no podamos determinar cuáles son los parámetros más relevantes y mejor explican la relación que hay en la naturaleza del problema. Por lo tanto podemos sacrificar algunos de ellos para determinar cuáles son las mejores variables "explicativas".



### One first solution (Backward stepwise selection)

Una vez que fiteamos todo el modelo podemos evaluar si el MLE de parámetros obtenidos es raro en relación a lo que nos presentan los datos, y en ese caso descartarlos. En detalle tenemos que un modelo lineal tiene como parámetros $\beta$ óptimos a:
$$
\beta_{MlE} = (X^{T} X)^{-1} X^{T} y\\
Var(\beta) = (X^{T}X)^{-1}\sigma^{2}
$$
donde $\sigma^{2}$ es la incerteza de los datos, que en caso de no conocerla podemos utilizar su MLE, $\sigma_{MLE} = \frac{1}{N-p-1} \sum_{i=1}^{N}(y_{i} - \bar{y})^{2}.$

Calculemos estos valores para una base de datos.


In [1]:
#### Imports

from numpy import *
from scipy.stats import binom,beta
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
from scipy.special import loggamma
import pandas as pd


In [2]:
df = pd.read_csv("../bin/data_sets/cancer.csv")
df_train = df[df["train"] == "T"]

In [3]:
df_train

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T
...,...,...,...,...,...,...,...,...,...,...
90,3.246491,4.101817,68,-1.386294,0,-1.386294,6,0,4.029806,T
91,2.532903,3.677566,61,1.348073,1,-1.386294,7,15,4.129551,T
92,2.830268,3.876396,68,-1.386294,1,1.321756,7,60,4.385147,T
93,3.821004,3.896909,44,-1.386294,1,2.169054,7,40,4.684443,T


## EDA

Estos datos buscan ver qué variables influyen en el cancer de prostata, las variables son: 
* log cancer volum : "lcavol"
* log prostate weight: "lweight"
* age
*  log of the amount of benign postatic hyperplasia: "lbph"
* seminal vesicle invasion: "svi"
* log of capsular penetration: "lcp"
* Gleadon score: "Gleason".
* Percent of Gleason scores 4 or 5: "ppgg5"

La variable apredecir es log of prostate-specific antigen: "lpsa"

Vemos de los datos que svi es binaria y que Gleason es una variable categórica ordenada.

In [4]:
X = df_train[['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']]
X = (X-X.mean())/X.std()
X["intercept"] = 1

Y = df_train["lpsa"]

In [5]:
aux_X = X.T.dot(X)
X_inv = pd.DataFrame(linalg.pinv(aux_X.values), aux_X.columns, aux_X.index)
beta_MLE = (X_inv.dot(X.T)).dot(Y)
var_data = 1/(len(Y)-len(X.T))*((Y-Y.mean())**2).sum()
std_betas = sqrt(X_inv*var_data)
std_diag = pd.DataFrame(diag(std_betas),std_betas.columns)

diag_X_inv = pd.DataFrame(diag(X_inv), X_inv.columns)


Z_score =pd.DataFrame(beta_MLE.values * 1/sqrt(var_data*diag_X_inv).values, index=diag_X_inv.index)
Z_score = pd.DataFrame(diag(Z_score), index = beta_MLE.index)


In [6]:
print("Los parametros valen: \n",beta_MLE)
print("Su std es: \n",std_diag)


Los parametros valen: 
 lcavol       0.716407
lweight      0.292642
age         -0.142550
lbph         0.212008
svi          0.309620
lcp         -0.289006
gleason     -0.020914
pgg45        0.277346
intercept    2.452345
dtype: float64
Su std es: 
                   0
lcavol     0.241484
lweight    0.192434
age        0.184719
lbph       0.186536
svi        0.226812
lcp        0.280017
gleason    0.257902
pgg45      0.288679
intercept  0.157405


Aquí ya calculamos las posteriors de los parametros, ahora lo que podemos hacer es ver cuáles de ellos podrían ser "0" con probabilidad razonable e ir tirandolos para quedarnos con un modelo que "necesite todos los parámetros. Para eso construimos el "Z-Score" el cual es una renormalización de cada parametro de la forma:
$$
z_{j} = \frac{\beta_{j}}{\sigma \sqrt{{X^{T}X}_{jj}}}
$$

Los $z_{j}$ son un escaleo de las distribuciones posterior de cada $\beta_{j}$ con el valor que toman podemos ver qué tan cerca de cero están y ver qué tan probable es que realmente sean cero.

In [7]:
Z_score

,0
lcavol,2.966684
lweight,1.520738
age,-0.771710
lbph,1.136548
svi,1.365096
lcp,-1.032099
gleason,-0.081091
pgg45,0.960742
intercept,15.579793


Los valores más chicos en módulo son los que serán menos relevantes, en este caso gelason age y pgg 45. De no estar satisfechos con el modelo los podemos remover y ver cómo se ven afectadas los parámetros de las otras variables.

### Implementemos lo mismo con Sklearn



In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
X = df_train[['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']]
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
reg = LinearRegression().fit(X, Y)

In [17]:
reg.coef_,reg.intercept_

(array([ 0.71104059,  0.29045029, -0.14148182,  0.21041951,  0.30730025,
        -0.28684075, -0.02075686,  0.27526843]),
 2.4523450850746262)

### Shrinage

#### Ridge regression

Descartar variables coincide con la idea de que alguno de sus parámetros sean cero y quedarnos con aquellas variables cuyos parámetros sean más significativos. Otra idea es hacer que los parámetros no puedan tomar valores muy grandes penalizando en la función de costo por su valor absoluto. Es decir que la loss function ahora será:

$$
\mathcal{L} = \left\{ \sum_{i=1}^{N}(y_{i} - \beta_{0} + \sum_{j=1}^{p} x_{ij}\beta_{j})^{2} + \lambda \sum_{j=1}^{p}\beta_{j}^{2}\right\}
$$

Esto es equivalente a proponer un hyperparámetro $\lambda$ que gobierna la prior de los $\beta$, es decir que la posterior de los $\beta$ es:

$$
P(\beta|D,\sigma^{2},\lambda) \propto P(D|\beta,\sigma^{2}\lambda) P(\beta|\sigma^{2}\lambda)  = \exp\left\{-\frac{1}{2\sigma^{2}}\sum_{i=1}^{N}(y_{i} - \beta_{0} + \sum_{j=1}^{p} x_{ij}\beta_{j})^{2}\right\} \exp\left\{-\frac{\lambda}{2\sigma^{2}} \sum_{j=1}^{p} \beta_{j}^{2}\right\}
$$

Hagamos el mismo fiteo pero con este hyperparámetro seteado a $1$

In [19]:
from sklearn.linear_model import Ridge
reg_Ridge = Ridge(alpha=1.0)
reg_Ridge = reg_Ridge.fit(X, Y)


$$



In [20]:
reg_Ridge.coef_,reg_Ridge.intercept_

(array([ 0.68540969,  0.28959545, -0.13430643,  0.20841057,  0.30162494,
        -0.25453234, -0.0112517 ,  0.25598543]),
 2.4523450850746262)

Los valores de los parámetros son todos más chicos, pero no en la misma proporción. La idea es que los que menos importen tomaran valores más chicos dejanod lugar a los más relevantes

### Lasso

Lasso propone lo miso que antes pero la unción de costo es $|\beta_{j}|$ en lugar del cuadrado

$$
P(\beta|D,\sigma^{2},\lambda) \propto P(D|\beta,\sigma^{2}\lambda) P(\beta|\sigma^{2}\lambda)  = \exp\left\{-\frac{1}{2\sigma^{2}}\sum_{i=1}^{N}(y_{i} - \beta_{0} + \sum_{j=1}^{p} x_{ij}\beta_{j})^{2}\right\} \exp\left\{-\frac{\lambda}{2\sigma^{2}} \sum_{j=1}^{p} |\beta_{j}|\right\}
$$


In [35]:
from sklearn.linear_model import Lasso
reg_Lasso = Lasso(alpha=0.1)
reg_Lasso = reg_Lasso.fit(X, Y)


In [36]:
reg_Lasso.coef_,reg_Lasso.intercept_

(array([ 0.57065706,  0.22862578, -0.        ,  0.10501276,  0.17097605,
         0.        ,  0.        ,  0.06532039]),
 2.4523450850746262)

Como es de esperar el efecto es más fuerte en el parámetro $\alpha$ o $\lambda$ dado que no tenemos potencia de $\beta$

### Observaciones

Esta frma de introducir un parámetro que llamaremos _hyperparámetro_ el cual regula el comportamiento de las variables que queremos estimar es una perilla que nos puede ayudar a fittear mejor nuestro modelo. Es el hecho de poner una variable que vincula a los parámetros entre sí por fuera de los datos que hace que la estructura del modelo sea más compleja y permita modelar los datos de una manera más interesante. 


### Priors parámetricas con interpretación. Modelos jerárquicos

##